In [ ]:
cd .. 

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import os

from neural_nets_library import training
from tree_to_sequence.tree_encoder import TreeEncoder
from tree_to_sequence.tree_decoder import TreeDecoder
from tree_to_sequence.grammar_tree_decoder import GrammarTreeDecoder
from tree_to_sequence.program_datasets import *
from tree_to_sequence.translating_trees import ( parent_to_category_FOR, category_to_child_FOR,
                                                 category_to_child_LAMBDA, 
                                                 parent_to_category_LAMBDA, ForGrammar, For,
                                                 LambdaGrammar, Lambda)
from tree_to_sequence.tree_to_tree_attention import TreeToTreeAttention
from functools import partial

In [ ]:
torch.cuda.set_device(1)

In [ ]:
num_vars = 10
num_ints = 11
one_hot = False
binarize = True
eos_token = False
long_base_case = True
input_as_seq = False
output_as_seq = False

In [ ]:
for_lambda_dset = ForLambdaDataset("ANC/VeryHard-arbitraryForList.json", binarize_input=binarize, binarize_output=False, 
                                   eos_token=eos_token, one_hot=one_hot, num_ints=num_ints, num_vars=num_vars,
                                   long_base_case=long_base_case, input_as_seq=input_as_seq, 
                                   output_as_seq=output_as_seq)
max_size = max([x[1].size() for x in for_lambda_dset])

In [ ]:
def reset_all_parameters_uniform(model, stdev):
    for param in model.parameters():
        nn.init.uniform_(param, -stdev, stdev)

In [ ]:
embedding_size = 256 #... 256 is from the paper, but 100 is WAY faster
hidden_size = 256
num_layers = 1
alignment_size = 50
align_type = 1
eos_bonus = 1 if eos_token else 0
encoder_input_size = num_vars + num_ints + len(for_ops) + eos_bonus
encoder = TreeEncoder(encoder_input_size, hidden_size, num_layers, [1, 2, 3, 4, 5], attention=True, one_hot=one_hot)
nclass = num_vars + num_ints + len(lambda_ops)
plot_every = 10
num_categories = len(LambdaGrammar)
num_possible_parents = len(Lambda)
save_file="test_save_file"
save_folder="test_save_folder"
save_every=20

decoder = GrammarTreeDecoder(embedding_size, hidden_size, num_categories, num_possible_parents, 
                             partial(parent_to_category_LAMBDA, num_vars, num_ints), 
                             partial(category_to_child_LAMBDA, num_vars, num_ints), 
                             share_linear=False, share_lstm_cell=False, num_ints_vars=num_ints + num_vars)
program_model = TreeToTreeAttention(encoder, decoder, hidden_size, embedding_size, nclass=nclass, root_value=nclass,
                                    alignment_size=alignment_size, align_type=align_type, max_size=max_size)
    
reset_all_parameters_uniform(program_model, 0.1)
encoder.initialize_forget_bias(3)
decoder.initialize_forget_bias(3)

In [ ]:
program_model = program_model.cuda()

In [ ]:
optimizer = optim.Adam(program_model.parameters(), lr=0.005)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=500, factor=0.8)

In [ ]:
# Counts the number of matches between the prediction and target.
def count_matches(prediction, target):
    matches = 0
    if int(prediction.value) == int(target.value):
        matches += 1
    for i in range(min(len(target.children), len(prediction.children))):
        matches += count_matches(prediction.children[i], target.children[i])
    return matches

# Program accuracy (1 if completely correct, 0 otherwise)
def program_accuracy(prediction, target):
    if prediction.size() == count_matches(prediction, target) and \
       prediction.size() == target.size():
        return 1
    else:
        return 0

# Calculate validation accuracy (this could either be program or token accuracy)
def validation_criterion(prediction, target):
    return program_accuracy(prediction, target)

In [ ]:
os.system("mkdir "+ save_folder)
best_model, train_plot_losses, validation_plot_losses = training.train_model_tree_to_tree(program_model, for_lambda_dset, 
                                optimizer, 
                                lr_scheduler=lr_scheduler, 
                                num_epochs=1, plot_every=plot_every,
                                batch_size=100, 
                                print_every=200, 
                                validation_criterion=validation_criterion,
                                use_cuda=True, 
                                deep_copy_desired=False,
                                plateau_lr=True,
                                save_file=save_file,
                                save_folder=save_folder,
                                save_every=save_every)

In [ ]:
plt.plot([x * plot_every for x in range(len(train_plot_losses))], train_plot_losses)
plt.title("Training Loss")
plt.xlabel('Training Examples')
plt.ylabel('Cross-Entropy Loss')
plt.savefig(save_folder + "/" + save_file + "_train_plot.png")
plt.close()

plt.plot([x * plot_every for x in range(len(validation_plot_losses))], validation_plot_losses)
plt.title("Training Program Accuracy")
plt.xlabel('Training Examples')
plt.ylabel('Percent Accurate Programs')
plt.savefig(save_folder + "/" + save_file + "_accuracy_plot.png")